In [8]:
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer

In [9]:
file_name = '../data/2015-06-01-2025-06-01-Europe.csv'

df = pd.read_csv(file_name, delimiter=',')
print(df.columns)
print(df)
df['tags'].unique()

Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'interaction', 'civilian_targeting', 'iso',
       'region', 'country', 'admin1', 'admin2', 'admin3', 'location',
       'latitude', 'longitude', 'geo_precision', 'source', 'source_scale',
       'notes', 'fatalities', 'tags', 'timestamp', 'population_best'],
      dtype='object')
       event_id_cnty       event_date  year  time_precision  \
0          UKR228116      23 May 2025  2025               1   
1          UKR228146      23 May 2025  2025               1   
2          UKR228615      23 May 2025  2025               1   
3          UKR228722      23 May 2025  2025               1   
4          UKR228830      23 May 2025  2025               1   
...              ...              ...   ...             ...   
621873          UKR3  01 January 2018  2018               1   
621874      UKR39146  01 January 2018  2018       

array([nan, 'crowd size=several', 'crowd size=no report', ...,
       'crowd size=at least 14,700', 'crowd size=90,000-400,000',
       'crowd size=about 36-40'], shape=(7104,), dtype=object)

In [10]:

df['event_type'].unique()
df = df.drop_duplicates(subset=['notes', 'event_date', 'country'])
print(df)

       event_id_cnty       event_date  year  time_precision  \
0          UKR228116      23 May 2025  2025               1   
1          UKR228146      23 May 2025  2025               1   
2          UKR228615      23 May 2025  2025               1   
3          UKR228722      23 May 2025  2025               1   
4          UKR228830      23 May 2025  2025               1   
...              ...              ...   ...             ...   
621872         UKR12  01 January 2018  2018               1   
621873          UKR3  01 January 2018  2018               1   
621874      UKR39146  01 January 2018  2018               1   
621875      UKR39147  01 January 2018  2018               1   
621876         UKR31  01 January 2018  2018               1   

             disorder_type                  event_type  \
0       Political violence  Explosions/Remote violence   
1       Political violence  Explosions/Remote violence   
2       Political violence  Explosions/Remote violence   
3       Pol

In [11]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
print(stopwords.words('english')[:10])
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def preprocess(text):
    text = re.sub(r'\W+', ' ', str(text).lower())
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)
regex_pattern =  r"^(?:\[)?.?\d{4}(?:[,.:]?\s)?"

df.notes = df['notes'].str.replace(regex_pattern, '', regex=True).str.strip()
df['clean_notes'] = df['notes'].apply(preprocess)
df['clean_notes'] = df['clean_notes'].apply(lambda x: ' '.join(str(x).split()[3:]))
df = df[df['clean_notes'].str.strip() != '']

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an']


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\semvv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\semvv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
ndf = df[df['event_type'].isin(['Protests', 'Riots'])] #maybe 'Violence against civilians'
ndf = ndf[ndf['sub_event_type'] != 'Mob violence']
print(ndf)

       event_id_cnty       event_date  year  time_precision   disorder_type  \
6            BEL4179      23 May 2025  2025               1  Demonstrations   
7            BEL4183      23 May 2025  2025               1  Demonstrations   
8            BGR4378      23 May 2025  2025               1  Demonstrations   
9            BGR4379      23 May 2025  2025               1  Demonstrations   
10           BGR4380      23 May 2025  2025               1  Demonstrations   
...              ...              ...   ...             ...             ...   
621871          UKR9  01 January 2018  2018               1  Demonstrations   
621872         UKR12  01 January 2018  2018               1  Demonstrations   
621873          UKR3  01 January 2018  2018               1  Demonstrations   
621874      UKR39146  01 January 2018  2018               1  Demonstrations   
621875      UKR39147  01 January 2018  2018               1  Demonstrations   

       event_type    sub_event_type                

In [13]:
ndf['country_code'] = ndf['event_id_cnty'].str[:3]

european_codes = [
    'ALB', 'AND', 'AUT', 'BEL', 'BGR', 'BIH', 'CHE', 'CYP', 'CZE', 'DEU', 'DNK',
    'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HRV', 'HUN', 'IRL', 'ISL', 'ITA',
    'KOS', 'LIE', 'LTU', 'LUX', 'LVA', 'MCO', 'MDA', 'MKD', 'MLT', 'MNE', 'NLD',
    'NOR', 'POL', 'PRT', 'ROU', 'SMR', 'SRB', 'SVK', 'SVN', 'SWE', 'VAT'
]

nnndf = ndf[ndf['country_code'].isin(european_codes)]
print(nnndf)


european_countries = [
    'Albania', 'Andorra', 'Austria', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria',
    'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
    'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kosovo',
    'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malta', 'Moldova',
    'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway', 'Poland',
    'Portugal', 'Romania', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia',
    'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'Vatican City'
]

nndf = ndf[ndf['country'].isin(european_countries)]
print(nndf)

       event_id_cnty       event_date  year  time_precision   disorder_type  \
6            BEL4179      23 May 2025  2025               1  Demonstrations   
7            BEL4183      23 May 2025  2025               1  Demonstrations   
8            BGR4378      23 May 2025  2025               1  Demonstrations   
9            BGR4379      23 May 2025  2025               1  Demonstrations   
10           BGR4380      23 May 2025  2025               1  Demonstrations   
...              ...              ...   ...             ...             ...   
621731          BGR9  04 January 2018  2018               1  Demonstrations   
621754          BGR5  04 January 2018  2018               3  Demonstrations   
621789          BGR1  03 January 2018  2018               1  Demonstrations   
621790       GRC2038  03 January 2018  2018               1  Demonstrations   
621817       GRC2057  02 January 2018  2018               1  Demonstrations   

       event_type    sub_event_type                

In [14]:
nndf.to_csv('../data/filtered_events.csv', index=False)
nnndf.to_csv('../data/filtered_events_country_code.csv', index=False)

In [16]:
nnndf['sub_event_type'].unique()

array(['Peaceful protest', 'Protest with intervention',
       'Violent demonstration', 'Excessive force against protesters'],
      dtype=object)